In [6]:
# Transformers installation
%pip install transformers datasets evaluate seqeval accelerate torch
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import torch

In [24]:
from huggingface_hub import notebook_login

notebook_login()

ImportError: The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywidgets` module: `pip install ipywidgets`.

In [ ]:
#Loading Data
import json
with open("data.json", encoding='utf-8') as fp:
    dataset = json.load(fp)

'''for i in range(len(dataset)):
    for j in range(len(dataset[i]['tokens'])):
        dataset[i]['tokens'][j] = dataset[i]['tokens'][j].upper()'''
dataset[0]

{'id': 0,
 'tokens': ['Abhishek',
  'Jha',
  'Application',
  'Development',
  'Associate',
  'Accenture',
  'Bengaluru,',
  'Karnataka',
  'Email',
  'Indeed',
  'indeed.com/r/Abhishek',
  'Jha/10e7a8cb732bc43a',
  'work',
  'organization',
  'provides',
  'opportunity',
  'improve',
  'skills',
  'knowledge',
  'individual',
  "company's",
  'growth',
  'best',
  'possible',
  'ways.',
  'Willing',
  'relocate',
  'Bangalore,',
  'Karnataka',
  'WORK',
  'EXPERIENCE',
  'Application',
  'Development',
  'Associate',
  'Accenture',
  'November',
  '2017',
  'Present',
  'Role',
  'Currently',
  'working',
  'Chat',
  'bot.',
  'Developing',
  'Backend',
  'Oracle',
  'PeopleSoft',
  'Queries',
  'Bot',
  'triggered',
  'based',
  'given',
  'input.',
  'Also,',
  'Training',
  'bot',
  'different',
  'possible',
  'utterances',
  'positive',
  'negative',
  ',',
  'given',
  'input',
  'user.',
  'EDUCATION',
  'B.E',
  'Information',
  'science',
  'engineering',
  'B.v.b',
  'colleg

In [19]:
weights = {
    "O": 0,
    "B-College Name": 0,
    "I-College Name": 0,
    "B-Skills": 0,
    "I-Skills": 0,
    "B-Graduation Year": 0,
    "I-Graduation Year": 0,
    "B-Degree": 0,
    "I-Degree": 0,
    "B-Years of Experience": 0,
    "I-Years of Experience": 0,
    "B-Companies worked at": 0,
    "I-Companies worked at": 0,
    "B-Designation": 0,
    "I-Designation": 0,
}
total=0

for i in dataset:
    for j in i['ner_tags']:
        if j not in weights.keys():
            continue
        weights[j]+=1
        total+=1

for k in weights:
    weights[k] /= total

In [20]:
weight_list = list(weights.values())
weight_list = [1-x for x in weight_list]

In [21]:
weight_list

[0.20485720441433464,
 0.9934646781413832,
 0.9912503291599837,
 0.9558327149115458,
 0.9091638139468078,
 0.9956909965767362,
 0.9998803054604649,
 0.9936322504967323,
 0.9946137457209202,
 0.9992938022167429,
 0.9996648552893017,
 0.9880784238623034,
 0.9955353936753405,
 0.9870969286381155,
 0.9919445574892873]

In [22]:

label2id = {
    "O": 0,
    "B-College Name": 1,
    "I-College Name": 2,
    "B-Skills": 3,
    "I-Skills": 4,
    "B-Graduation Year": 5,
    "I-Graduation Year": 6,
    "B-Degree": 7,
    "I-Degree": 8,
    "B-Years of Experience": 9,
    "I-Years of Experience": 10,
    "B-Companies worked at": 11,
    "I-Companies worked at": 12,
    "B-Designation": 13,
    "I-Designation": 14,
}

id2label = {
    0: "O",
    1: "B-College Name",
    2: "I-College Name",
    3: "B-Skills",
    4: "I-Skills",
    5: "B-Graduation Year",
    6: "I-Graduation Year",
    7: "B-Degree",
    8: "I-Degree",
    9: "B-Years of Experience",
    10: "I-Years of Experience",
    11: "B-Companies worked at",
    12: "I-Companies worked at",
    13: "B-Designation",
    14: "I-Designation",
}

In [23]:
for i in range(len(dataset)):
    dataset[i]["ner_tags"] = [label2id[j] for j in dataset[i]["ner_tags"]]

dataset[0]

KeyError: 'B-UNKNOWN'

In [12]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(dataset,test_size=0.15)
print(len(train_set),len(test_set))

3725 658


In [13]:
from datasets import IterableDataset

def gen_train():
    for i in train_set:
        yield i

def gen_test():
    for i in test_set:
        yield i

train_dataset = IterableDataset.from_generator(gen_train)
test_dataset = IterableDataset.from_generator(gen_test)

In [14]:
label_list = list(label2id.keys())
label_list

['O',
 'B-no',
 'I-no',
 'B-class',
 'I-class',
 'B-method',
 'I-method',
 'B-attribute',
 'I-attribute',
 'B-association',
 'I-association',
 'B-generalization',
 'I-generalization']

In [41]:
# pretrained_model = "bert-large-uncased"
pretrained_model = "roberta-large"

In [44]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(pretrained_model, add_prefix_space=True)

In [45]:
# example = wnut["train"][0]
example = dataset[0]
print(example)
# tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

{'id': '0', 'ner_tags': [1, 5, 0, 5, 3, 0], 'tokens': ['3.1.1', 'Start', 'and', 'Stop', 'Diagnostics', 'Use Case']}


['<s>',
 'Ġ3',
 '.',
 '1',
 '.',
 '1',
 'ĠStart',
 'Ġand',
 'ĠStop',
 'ĠDiagn',
 'ostics',
 'ĠUse',
 'ĠCase',
 '</s>']

In [46]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [47]:
# tokenized_dataset = list(map(tokenize_and_align_labels, dataset))
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

In [48]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [49]:
import evaluate

seqeval = evaluate.load("seqeval")

In [50]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [51]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    pretrained_model, num_labels=13, id2label=id2label, label2id=label2id)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
from torch import nn

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get('logits')
        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(weight_list, device='cuda:0'))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [64]:
batch_size = 8

training_args = TrainingArguments(
    output_dir="model",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    max_steps = 1000,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub = True,
    fp16=True,  # Gives Low Precision
    # optim = "adamw_bnb_8bit"
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [65]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,No log,0.551265,0.538939,0.735812,0.622173,0.878719
1,0.404100,0.517935,0.539778,0.761252,0.631664,0.879746
2,0.396800,0.534966,0.593176,0.737117,0.657359,0.896676


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1000, training_loss=0.4004513244628906, metrics={'train_runtime': 1388.3042, 'train_samples_per_second': 5.762, 'train_steps_per_second': 0.72, 'total_flos': 883802868937644.0, 'train_loss': 0.4004513244628906, 'epoch': 2.07})

In [66]:
from transformers import pipeline

classifier = pipeline(task="ner",device='cuda:0',model=trainer.model,tokenizer=trainer.tokenizer,aggregation_strategy = "simple")
text = "The device beeps and the message &quot;INVALID INPUT&quot is displayed on the screen ."
classifier(text)

[{'entity_group': 'class',
  'score': 0.99917704,
  'word': ' device',
  'start': 4,
  'end': 10},
 {'entity_group': 'method',
  'score': 0.73989403,
  'word': ' be',
  'start': 11,
  'end': 13},
 {'entity_group': 'method',
  'score': 0.6233481,
  'word': 'eps',
  'start': 13,
  'end': 16},
 {'entity_group': 'method',
  'score': 0.7253102,
  'word': ' displayed',
  'start': 61,
  'end': 70}]

In [67]:
trainer.push_to_hub()

'https://huggingface.co/suyashmittal/model/tree/main/'